# Train a Convolutional Variational Autoencoder 

Using the Conv-VAE package to train a convolutional variational autoencoder. The parameters of the training are describe by an experiment of type "conv_vae". 

__Note:__ The result of running this code is the creation of a "final model". The encoder portion of this model is the component that can be used for the encoder of the robot. Once the training had been done, the subdir of this experiment and the checkpoint we want to use will need to updated into the experiment run-specific file in the model_subdir and model_checkpoint fields.


In [1]:
import sys
sys.path.append("..")
from settings import Config
import pathlib

# adding the Julian-8897-Conv-VAE-PyTorch into the path
sys.path.append(Config()["conv_vae"]["code_dir"])

# Oh, this hack was fixing something, but for me it is the other way around
#temp = pathlib.PosixPath
#pathlib.PosixPath = pathlib.WindowsPath

from conv_vae import get_conv_vae_config, create_configured_vae_json, train,  latest_training_run

Loading pointer config file: /home/lboloni/.config/BerryPicker/mainsettings.yaml
Loading machine-specific config file: /home/lboloni/Insync/lotzi.boloni@gmail.com/Google Drive/LotziStudy/Code/PackageTracking/BerryPicker/settings/settings-tredy2.yaml


## Setting up the parameters based on the experiment

The parameters of the training process are set up in the j

This will give an error about the pandas, which is due to certain adressing mode in the Conv-Vae util.py, which probably will stop working in pandas 3.0

In [2]:
# run = "vae_00"
run = "vae_01"
exp = Config().get_experiment("conv_vae", run)

file = create_configured_vae_json(exp)
print(file)
vae_config = get_conv_vae_config(file)
# pretty printing the dict behind the config
# print(json.dumps(config.config, indent=4)) 

Missing experiment system dependent config file /home/lboloni/Insync/lotzi.boloni@gmail.com/Google Drive/LotziStudy/Code/PackageTracking/BerryPicker/settings/experiment-config/Tredy2/conv_vae/vae_01_sysdep.yaml, that is ok, proceeding.
Configuration for experiment: conv_vae/vae_01 successfully loaded
/home/lboloni/Documents/Hackingwork/_Checkouts/BerryPicker/BerryPicker/src/encoding_conv_vae/conv-vae-config-default.json
{'name': 'VAE_Robot', 'n_gpu': 1, 'arch': {'type': 'VanillaVAE', 'args': {'in_channels': 3, 'latent_dims': 256, 'flow': False}}, 'data_loader': {'type': 'CelebDataLoader', 'args': {'data_dir': '/home/lboloni/Documents/Hackingwork/__Temporary/BerryPicker-experiments/conv_vae/vae_01/vae-training-data', 'batch_size': 64, 'shuffle': True, 'validation_split': 0.2, 'num_workers': 2}}, 'optimizer': {'type': 'Adam', 'args': {'lr': 0.005, 'weight_decay': 0.0, 'amsgrad': True}}, 'loss': 'elbo_loss', 'metrics': [], 'lr_scheduler': {'type': 'StepLR', 'args': {'step_size': 50, 'gamm

In [3]:
# update the vae_config based on parameters from the experiment
# vae_config["trainer"]["epochs"] = exp["epochs"] # was 100
# vae_config["trainer"]["save_period"] = exp["save_period"] # 5
# vae_config["arch"]["args"]["latent_dims"] = exp["latent_dims"]

# FIXME: among other things, the problem is that this is not being written back into the config file 

# print(exp["latent_dims"])

In [4]:
# actually run the training
print(f'Running the trainer from scratch for {vae_config["trainer"]["epochs"]}')
trainer = train(vae_config)

Running the trainer from scratch for 100


/home/lboloni/Documents/Hackingwork/_Checkouts/Julian-8897-Conv-VAE-PyTorch/Conv-VAE-PyTorch/utils/util.py:59: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  self._data.total[key] += value * n
/home/lboloni/Documents/Hackingwork/_Checkouts/Ju

In [ ]:
# These are the metrics recorded
# they are of utils/util.py / MetricTracker which has a pandas dataframe as data
print(trainer.train_metrics)
print(trainer.valid_metrics)
# 
trainer.train_metrics._data
# trainer.valid_metrics._data